- 입력 : line, current_station, next_station, arriving_time, exit
- 출력 : 가장 빠른 지하철 시간

In [2]:
import pandas as pd

In [3]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [4]:
con1 = sqlite3.connect('subway_timetable.db')
con2 = sqlite3.connect('subway_inner_path.db') 

In [25]:
df1 = pd.read_sql_query("SELECT * FROM subway_timetable", con1)
df2 = pd.read_sql_query("SELECT * FROM subway_inner_path", con2)

In [52]:
# line code 받음

def subway_departure_time(line_code, current_station, next_station, for_hour, for_minute, exit_code):
    # line과 current_station에 승강기 이용 가능한지 탐색
    line_dic = {'1':'1호선', '2':'2호선', '3':'3호선', '4':'4호선', '5':'5호선', '6':'6호선', 
                '7':'7호선', '8':'8호선', '9':'9호선','K':'경의선/중앙선', 'B':'분당선/수인선', 
                'G':'경춘선', 'S':'신분당선', 'A':'공항철도', 'SU':'분당선/수인선'}

    ele = 0
    if (line_dic[line_code] in df2.line.unique()) and (current_station in df2.station.unique()):
        for i in df2[df2.line==line_dic[line_code]][df2.station == current_station].inner_path:
            if line_dic[exit_code] in i:
                ele = 1
                break
                
        # 만약 해당 출구에 승강기가 없는 경우, 승강기가 있는 출구로 반환
        if ele == 0:
            for_exit = i.split(' ↔ ')[-1]
        
        # 승강장에 도착한 시간 = 도착시간 + 이동시간
        moving_time = int(df2[df2.line==line_dic[line_code]][df2.station == current_station][df2.inner_path == i].time)
        for_time = for_minute + moving_time
        
        _hour = int(for_time/60) + for_hour
        _minute = for_time % 60
        
        # 승강장에 도착한 시간에서 가장 빨리오는 지하철 시간 반환
        temp = df1[df1.line == line_dic[line_code]][df1.station == current_station][df1.next_station == next_station]
        hour_list = list(temp.iloc[:, 7])
        minute_list = list(temp.iloc[:, 8])
        
        for k in range(len(hour_list)):
            if hour_list[k] >= _hour:
                if minute_list[k] >= _minute:
                    break
        
        if minute_list[k] < _minute:
            waiting_time = 60 + minute_list[k] - _minute
        else:
            waiting_time = minute_list[k] - _minute
            
        for_way_ = df2[df2.line==line_dic[line_code]][df2.station == current_station][df2.inner_path == i].facilities
        for l in for_way_:
            for_way = str(l).split(' ↔ ')
        for_inner_ = i.split(' ↔ ')
        
        output_path = []
        for cnt in range(len(for_inner_)-1):
            output = ""
            output += (for_inner_[cnt] +"부터 " + for_inner_[cnt+1] + "까지 " + for_way[cnt] + "로 이동") 
            output_path.append(output)
            
        return for_exit, moving_time, waiting_time, str(hour_list[k]) + "시 " + str(minute_list[k]) + "분", output_path
    else:
        return False

In [53]:
subway_departure_time('6', '상수', '광흥창', 6, 15, '2')

C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


('4번 출구',
 18,
 7,
 '6시 40분',
 ['승강장부터 지하 2층까지 엘리베이터로 이동', '지하 2층부터 4번 출구까지 휠체어 리프트로 이동'])

---